# Performance Table

In [1]:
import pandas as pd
import numpy as np
import os
import sys


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
import os
os.getcwd()
os.chdir('../')


from utils.utils     import *
from utils.constants import *
from utils.metrics   import *

from os import listdir
from os.path import isfile, join

In [3]:
path_ = os.getcwd()

# benchmark

In [4]:
y_train = pd.read_csv(path_ + '\\data' + '\\y_train.csv')

y_test = pd.read_csv(path_ + '\\data' + '\\y_test.csv')

In [5]:
y_predict_average_ini = [1] * len(y_test)

mean_predict = (np.mean(y_train))[0]
y_predict_average = [ x * mean_predict  for x in y_predict_average_ini ]

D:\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [6]:
benchmark_results = pd.concat([y_test,pd.DataFrame({'forecast': y_predict_average})], axis=1)
benchmark_results = benchmark_results.rename(columns={"measurement": "actual"})

In [7]:
benchmark_df = compute_metrics(benchmark_results, EVAL_METRIC_LIST)

In [8]:
benchmark_summary =  (pd.DataFrame(benchmark_df, index = ['benchmark'])*1000).round(3)

In [9]:
benchmark_summary

,rmse,mape,wmape,wbias,wuforec,woforec
benchmark,29.518,32.206,32.35,-3.256,14.547,17.803


# Models Summary

In [10]:
def model_summary(path_, rel_path):
    """
    path_: path for 'water-quality-prediction-08-27-yl-rerun' or similar folder
    
    rel_path: path for saved best models
    
    Returns:
    Two Pd.DataFrame()
    
    performance_summary: eval_metrics summary
    timing_summary:
    
    """
        
    performance_summary = pd.DataFrame()
    timing_summary = pd.DataFrame()

    results_path = path_ + "\\" + rel_path
    results_files = [f for f in listdir(results_path) if isfile(join(results_path, f))]
    filtered_results_files = [word for word in results_files if ".joblib" in word]
    
    for best_model_name in filtered_results_files:
        best_model_file = path_ + "\\" + rel_path + "\\" + best_model_name
        best_model, best_model_info = load_model(best_model_file)
        eval_metrics= best_model_info['eval_metrics']
        eval_metrics = (pd.DataFrame(eval_metrics, index = [best_model_name])*1000).round(3)
        performance_summary = pd.concat([performance_summary, eval_metrics])

        timing = {k:best_model_info[k] for k in ('totalling_fits', 'cv', 'tuning_time',
     'best_model_fitting_time') if k in best_model_info}
        timing['average_tuning'] = timing['tuning_time'] / timing['totalling_fits']

        timing = (pd.DataFrame(timing, index = [best_model_name])).round(3)
        timing_summary = pd.concat([timing_summary, timing])
    return performance_summary, timing_summary
    

# cv5_numerical only

In [11]:
rel_path = 'results\\cv5_numerical only'

In [12]:
performance_summary_numerical_only, timing_summary_numerical_only = model_summary(path_, rel_path)

In [13]:
performance_summary_numerical_only = performance_summary_numerical_only.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_28464\3348829722.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_only = performance_summary_numerical_only.append(benchmark_summary)


In [14]:
performance_summary_numerical_only

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,10.882,9.336,9.476,0.919,5.198,4.278
best_lin_model.joblib,12.035,10.724,10.898,0.409,5.654,5.244
best_mlp_model.joblib,11.619,9.996,10.151,0.114,5.132,5.018
best_rf_model.joblib,11.473,9.711,9.866,0.824,5.345,4.521
best_xgb_model.joblib,10.956,9.227,9.368,0.982,5.175,4.193
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [15]:
performance_summary_numerical_only.style.to_latex()

'\\begin{tabular}{lrrrrrr}\n & rmse & mape & wmape & wbias & wuforec & woforec \\\\\nbest_lightGBM_model.joblib & 10.882000 & 9.336000 & 9.476000 & 0.919000 & 5.198000 & 4.278000 \\\\\nbest_lin_model.joblib & 12.035000 & 10.724000 & 10.898000 & 0.409000 & 5.654000 & 5.244000 \\\\\nbest_mlp_model.joblib & 11.619000 & 9.996000 & 10.151000 & 0.114000 & 5.132000 & 5.018000 \\\\\nbest_rf_model.joblib & 11.473000 & 9.711000 & 9.866000 & 0.824000 & 5.345000 & 4.521000 \\\\\nbest_xgb_model.joblib & 10.956000 & 9.227000 & 9.368000 & 0.982000 & 5.175000 & 4.193000 \\\\\nbenchmark & 29.518000 & 32.206000 & 32.350000 & -3.256000 & 14.547000 & 17.803000 \\\\\n\\end{tabular}\n'

In [16]:
timing_summary_numerical_only

,totalling_fits,cv,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,5,152.820,0.188,0.076
best_lin_model.joblib,150,5,8.794,0.079,0.059
best_mlp_model.joblib,40,5,1974.694,218.523,49.367
best_rf_model.joblib,1200,5,1955.061,65.293,1.629
best_xgb_model.joblib,5760,5,2427.986,4.314,0.422


In [17]:
timing_summary_numerical_only.style.to_latex()

'\\begin{tabular}{lrrrrr}\n & totalling_fits & cv & tuning_time & best_model_fitting_time & average_tuning \\\\\nbest_lightGBM_model.joblib & 2000 & 5 & 152.820000 & 0.188000 & 0.076000 \\\\\nbest_lin_model.joblib & 150 & 5 & 8.794000 & 0.079000 & 0.059000 \\\\\nbest_mlp_model.joblib & 40 & 5 & 1974.694000 & 218.523000 & 49.367000 \\\\\nbest_rf_model.joblib & 1200 & 5 & 1955.061000 & 65.293000 & 1.629000 \\\\\nbest_xgb_model.joblib & 5760 & 5 & 2427.986000 & 4.314000 & 0.422000 \\\\\n\\end{tabular}\n'

# cv5_numerical_only_withstd

In [18]:
rel_path = 'results\\cv5_numerical_only_withstd'

In [19]:
performance_summary_numerical_cv5_numerical_only_withstd, timing_summary_numerical_cv5_numerical_only_withstd = model_summary(path_, rel_path)

In [20]:
performance_summary_numerical_cv5_numerical_only_withstd = performance_summary_numerical_cv5_numerical_only_withstd.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_28464\4054605800.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_cv5_numerical_only_withstd = performance_summary_numerical_cv5_numerical_only_withstd.append(benchmark_summary)


In [21]:
performance_summary_numerical_cv5_numerical_only_withstd

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,10.844,9.327,9.465,0.862,5.164,4.302
best_lin_model.joblib,12.037,10.742,10.916,0.406,5.661,5.255
best_mlp_model.joblib,14.198,12.919,13.161,-5.499,3.831,9.330
best_rf_model.joblib,11.472,9.710,9.865,0.837,5.351,4.514
best_xgb_model.joblib,10.958,9.221,9.363,0.985,5.174,4.189
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [22]:
performance_summary_numerical_cv5_numerical_only_withstd.style.to_latex()

'\\begin{tabular}{lrrrrrr}\n & rmse & mape & wmape & wbias & wuforec & woforec \\\\\nbest_lightGBM_model.joblib & 10.844000 & 9.327000 & 9.465000 & 0.862000 & 5.164000 & 4.302000 \\\\\nbest_lin_model.joblib & 12.037000 & 10.742000 & 10.916000 & 0.406000 & 5.661000 & 5.255000 \\\\\nbest_mlp_model.joblib & 14.198000 & 12.919000 & 13.161000 & -5.499000 & 3.831000 & 9.330000 \\\\\nbest_rf_model.joblib & 11.472000 & 9.710000 & 9.865000 & 0.837000 & 5.351000 & 4.514000 \\\\\nbest_xgb_model.joblib & 10.958000 & 9.221000 & 9.363000 & 0.985000 & 5.174000 & 4.189000 \\\\\nbenchmark & 29.518000 & 32.206000 & 32.350000 & -3.256000 & 14.547000 & 17.803000 \\\\\n\\end{tabular}\n'

In [23]:
timing_summary_numerical_cv5_numerical_only_withstd

,totalling_fits,cv,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,5,159.920,0.193,0.080
best_lin_model.joblib,150,5,3.323,0.071,0.022
best_mlp_model.joblib,40,5,1948.634,116.050,48.716
best_rf_model.joblib,1200,5,1934.507,64.575,1.612
best_xgb_model.joblib,5760,5,2429.882,4.297,0.422


In [24]:
timing_summary_numerical_cv5_numerical_only_withstd.style.to_latex()

'\\begin{tabular}{lrrrrr}\n & totalling_fits & cv & tuning_time & best_model_fitting_time & average_tuning \\\\\nbest_lightGBM_model.joblib & 2000 & 5 & 159.920000 & 0.193000 & 0.080000 \\\\\nbest_lin_model.joblib & 150 & 5 & 3.323000 & 0.071000 & 0.022000 \\\\\nbest_mlp_model.joblib & 40 & 5 & 1948.634000 & 116.050000 & 48.716000 \\\\\nbest_rf_model.joblib & 1200 & 5 & 1934.507000 & 64.575000 & 1.612000 \\\\\nbest_xgb_model.joblib & 5760 & 5 & 2429.882000 & 4.297000 & 0.422000 \\\\\n\\end{tabular}\n'

# cv5_withCategorical_withstd

In [25]:
rel_path = 'results\\cv5_withCategorical_withstd'

In [26]:
performance_summary_numerical_cv5_withCategorical_withstd, timing_summary_numerical_cv5_withCategorical_withstd = model_summary(path_, rel_path)


In [27]:
performance_summary_numerical_cv5_withCategorical_withstd = performance_summary_numerical_cv5_withCategorical_withstd.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_28464\3223384742.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_cv5_withCategorical_withstd = performance_summary_numerical_cv5_withCategorical_withstd.append(benchmark_summary)


In [28]:
performance_summary_numerical_cv5_withCategorical_withstd

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,11.040,9.218,9.358,0.947,5.152,4.205
best_lin_model.joblib,11.914,10.792,10.958,1.315,6.136,4.822
best_mlp_model.joblib,14.420,14.541,14.608,3.970,9.289,5.319
best_rf_model.joblib,11.711,9.705,9.874,0.653,5.263,4.611
best_xgb_model.joblib,11.139,9.288,9.435,1.122,5.279,4.157
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [29]:
performance_summary_numerical_cv5_withCategorical_withstd.style.to_latex()

'\\begin{tabular}{lrrrrrr}\n & rmse & mape & wmape & wbias & wuforec & woforec \\\\\nbest_lightGBM_model.joblib & 11.040000 & 9.218000 & 9.358000 & 0.947000 & 5.152000 & 4.205000 \\\\\nbest_lin_model.joblib & 11.914000 & 10.792000 & 10.958000 & 1.315000 & 6.136000 & 4.822000 \\\\\nbest_mlp_model.joblib & 14.420000 & 14.541000 & 14.608000 & 3.970000 & 9.289000 & 5.319000 \\\\\nbest_rf_model.joblib & 11.711000 & 9.705000 & 9.874000 & 0.653000 & 5.263000 & 4.611000 \\\\\nbest_xgb_model.joblib & 11.139000 & 9.288000 & 9.435000 & 1.122000 & 5.279000 & 4.157000 \\\\\nbenchmark & 29.518000 & 32.206000 & 32.350000 & -3.256000 & 14.547000 & 17.803000 \\\\\n\\end{tabular}\n'

In [30]:
timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits'] = timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits'] * 5 

In [31]:
timing_summary_numerical_cv5_withCategorical_withstd['average_tuning'] = timing_summary_numerical_cv5_withCategorical_withstd['tuning_time']/ timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits']

In [32]:
timing_summary_numerical_cv5_withCategorical_withstd.round(3)

,totalling_fits,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,213.437,0.463,0.107
best_lin_model.joblib,150,44.390,0.825,0.296
best_mlp_model.joblib,40,2147.055,65.814,53.676
best_rf_model.joblib,1200,3633.047,115.665,3.028
best_xgb_model.joblib,5760,13320.127,4.156,2.313


In [33]:
timing_summary_numerical_cv5_withCategorical_withstd.style.to_latex()

'\\begin{tabular}{lrrrr}\n & totalling_fits & tuning_time & best_model_fitting_time & average_tuning \\\\\nbest_lightGBM_model.joblib & 2000 & 213.437000 & 0.463000 & 0.106719 \\\\\nbest_lin_model.joblib & 150 & 44.390000 & 0.825000 & 0.295933 \\\\\nbest_mlp_model.joblib & 40 & 2147.055000 & 65.814000 & 53.676375 \\\\\nbest_rf_model.joblib & 1200 & 3633.047000 & 115.665000 & 3.027539 \\\\\nbest_xgb_model.joblib & 5760 & 13320.127000 & 4.156000 & 2.312522 \\\\\n\\end{tabular}\n'